In [1]:
import numpy as np
import pandas as pd
import os
import math
import cv2
from tqdm import tqdm

from keras.utils import Sequence
from keras.layers import Input
from keras.models import Model, load_model
from keras.layers import Flatten, Dense, Activation, Dropout, Reshape
from keras.layers import BatchNormalization
from keras.layers import Conv3D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow as tf

%pylab inline

Using TensorFlow backend.


Populating the interactive namespace from numpy and matplotlib


# Loading Complete Data

In [2]:
%%time
initial_path = './images/'
X = []
for directory in sorted(os.listdir(initial_path)):
    path = initial_path + directory + '/Camera/RGB/'
    for image in sorted(os.listdir(path)):
        X.append(os.path.abspath(path+image))
print (len(X))

432000
CPU times: user 2.87 s, sys: 418 ms, total: 3.29 s
Wall time: 6.01 s


### Reading labels

In [3]:
labels = pd.read_csv("target_values.csv", header=None, names=['frame','steer','throttle','brake','speed'])
labels.drop(['frame'],axis=1,inplace=True)
print (labels.shape)
labels.head()

(432000, 4)


,steer,throttle,brake,speed
0,-0.000238,1.0,0.0,0.000000e+00
1,-0.000239,1.0,0.0,2.548699e-09
2,-0.000239,1.0,0.0,5.095700e-09
3,-0.000239,1.0,0.0,7.641002e-09
4,-0.000239,1.0,0.0,1.018461e-08


### Reading all the images

In [5]:
x_train = np.zeros((432000,88,200,3),dtype='uint8')

In [9]:
# x_train = np.array([ cv2.cvtColor(cv2.imread(x),cv2.COLOR_BGR2RGB)[30:-22,:] for x in tqdm(X)], dtype='uint8')
for idx,x in enumerate(tqdm(X)):
    x_train[idx] = cv2.cvtColor(cv2.imread(x),cv2.COLOR_BGR2RGB)[30:-22,:]
    

100%|██████████| 432000/432000 [13:25<00:00, 536.14it/s]


### Shaping Data For 3D Convs

In [ ]:
indexes = np.zeros((batch_size,time),dtype=np.int)
for i in range(batch_size):
    indexes[i] = np.arange(0+i,time+i)
indexes = indexes.flatten()

In [57]:
model = load_model('init_model_softmax.h5')

In [2]:
def conv3d_layer(x_input, filters, kernel_size, strides = 1, padding = 'SAME', dropout = 0.0):
    conv_out = Conv3D(filters = filters, kernel_size = kernel_size, strides = strides, padding = padding)(x_input)
    bn_out = BatchNormalization()(conv_out)
#     dropout_out = Dropout(dropout)(bn_out)
    return Activation('relu')(bn_out)

def Dense_layer(x_input, units, dropout = 0.0):
    dense_out = Dense(units = units)(x_input)
    bn_out = BatchNormalization()(dense_out)
    dropout_out = Dropout(dropout)(bn_out)
    return Activation('relu')(dropout_out)

In [4]:
# # define PilotNet model, with batch normalization included.
def get_Internals(image,speed):
    x = BatchNormalization()(image)
    x = conv3d_layer(x,24, kernel_size=5, strides=(1,2,2))
    x = conv3d_layer(x,36, kernel_size=5, strides=(1,2,2))
    x = conv3d_layer(x,48, kernel_size=5, strides=(1,2,2))
    x = conv3d_layer(x,64, kernel_size=3, strides=1, padding = 'valid')
    x = conv3d_layer(x,64, kernel_size=3, strides=1, padding = 'valid')

    img_conv = Flatten()(x)
    
    ## pass speed through some fully connected layers
    speed_dense = Dense(128)(speed)
    speed_dense = BatchNormalization()(speed_dense)
    speed_dense = Activation('relu')(speed_dense)
    
    speed_dense = Dense(128)(speed_dense)
    speed_dense = BatchNormalization()(speed_dense)
    speed_dense = Activation('relu')(speed_dense)
    
#     print (speed_dense.shape,img_conv.shape,x.shape)
    ## Concatinating both speed and extracted images
    x = concatenate([img_conv,speed_dense])
    
    # Steering
    x_steer = Dense(100)(x)
    x_steer = BatchNormalization()(x_steer)
    x_steer = Activation('relu')(x_steer)
    
    x_steer = Dense(50)(x_steer)
    x_steer = BatchNormalization()(x_steer)
    x_steer = Activation('relu')(x_steer)
    
    x_steer = Dense(10, activation='relu')(x_steer)
    x_steer = BatchNormalization()(x_steer)
    x_steer = Activation('relu')(x_steer)
    
    steering = Dense(1, name='steering')(x_steer)
    
    # Brake
    x_brake = Dense(100)(x)
    x_brake = BatchNormalization()(x_brake)
    x_brake = Activation('relu')(x_brake)
    
    x_brake = Dense(50)(x_brake)
    x_brake = BatchNormalization()(x_brake)
    x_brake = Activation('relu')(x_brake)
    
    x_brake = Dense(10)(x_brake)
    x_brake = BatchNormalization()(x_brake)
    x_brake = Activation('relu')(x_brake)
    
    brake = Dense(2, name='brake', activation = 'softmax')(x_brake)
    
    # Throttle
    x_throttle = Dense(100)(x)
    x_throttle = BatchNormalization()(x_throttle)
    x_throttle = Activation('relu')(x_throttle)
    
    x_throttle = Dense(50)(x_throttle)
    x_throttle = BatchNormalization()(x_throttle)
    x_throttle = Activation('relu')(x_throttle)
    
    x_throttle = Dense(10)(x_throttle)
    x_throttle = BatchNormalization()(x_throttle)
    x_throttle = Activation('relu')(x_throttle)
    
    throttle = Dense(3, name='throttle', activation = 'softmax')(x_throttle)
    
    return steering,throttle,brake 

inp_rgb = Input(shape=(9,88,200,3), name='rgb')
inp_speed = Input(shape=(1,), name= 'speed')
steering,throttle,brake = get_Internals(inp_rgb,inp_speed)

model = Model(inputs=[inp_rgb,inp_speed], outputs=[steering,throttle,brake])
adam = Adam(lr = 1e-3)

model.compile(optimizer=adam, loss={'steering':'mse','throttle':'sparse_categorical_crossentropy',
                                    'brake':'sparse_categorical_crossentropy'},
              metrics={'steering':'mae', 'throttle':'accuracy','brake':'accuracy'}) 
model.summary()

ValueError: zero-dimensional arrays cannot be concatenated

In [13]:
batch_size = 1024
total_images = 432000
num_batches = int(math.ceil(total_images/batch_size))
print (num_batches)

421


In [21]:
model_filename= os.path.join('model_checkpoints', 'model.{epoch:03d}-{loss:.4f}.h5')
checkpoint = ModelCheckpoint(model_filename,monitor='loss',verbose=0,save_best_only=True)
stopping = EarlyStopping(monitor='loss', min_delta=0.00015, patience=10, verbose=1, mode='min')

In [29]:
labels.iloc[(labels.iloc[:,1]==0.5).values,1]=2

In [37]:
labels.throttle = labels.throttle.astype(int)
labels.brake = labels.brake.astype(int)

In [58]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
rgb (InputLayer)                (None, 88, 200, 3)   0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 88, 200, 3)   12          rgb[0][0]                        
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 42, 98, 24)   1824        batch_normalization_1[0][0]      
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 42, 98, 24)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
dropout_1 

In [38]:
labels.head()

,steer,throttle,brake,speed
0,-0.000238,1,0,0.000000e+00
1,-0.000239,1,0,2.548699e-09
2,-0.000239,1,0,5.095700e-09
3,-0.000239,1,0,7.641002e-09
4,-0.000239,1,0,1.018461e-08


In [ ]:
history = model.fit({'rgb':x_train, 'speed':labels.speed.values.reshape((-1,1))},
                    {'steering':labels.steer.values.reshape((-1,1))*3.0,'throttle':labels.throttle.values.reshape((-1,1)),
                     'brake':labels.brake.values.reshape((-1,1))}, batch_size=batch_size, epochs=100, callbacks=[checkpoint, stopping])

Epoch 1/100
432000/432000 [==============================] - 492s 1ms/step - loss: 0.2594 - steering_loss: 0.0131 - throttle_loss: 0.1585 - brake_loss: 0.0878 - steering_mean_absolute_error: 0.0529 - throttle_acc: 0.9417 - brake_acc: 0.9683
Epoch 2/100
432000/432000 [==============================] - 492s 1ms/step - loss: 0.1748 - steering_loss: 0.0082 - throttle_loss: 0.1063 - brake_loss: 0.0603 - steering_mean_absolute_error: 0.0367 - throttle_acc: 0.9613 - brake_acc: 0.9791
Epoch 3/100
432000/432000 [==============================] - 491s 1ms/step - loss: 0.1378 - steering_loss: 0.0063 - throttle_loss: 0.0834 - brake_loss: 0.0482 - steering_mean_absolute_error: 0.0303 - throttle_acc: 0.9700 - brake_acc: 0.9836
Epoch 4/100
198656/432000 [============>.................] - ETA: 4:26 - loss: 0.1154 - steering_loss: 0.0054 - throttle_loss: 0.0699 - brake_loss: 0.0401 - steering_mean_absolute_error: 0.0277 - throttle_acc: 0.9749 - brake_acc: 0.9868

In [ ]:
plt.plot(np.arange(len(history.history['loss'])),history.history['loss'],color= 'r',label = 'Loss')
plt.legend()

In [ ]:
plt.plot(np.arange(len(history.history['loss'])),history.history['mean_absolute_error'],color= 'b',label = 'Mean Absolute Error')
plt.legend()